In [2]:
import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.WARNING)
logging.root.level = logging.WARNING

import os
import jieba
import codecs
import time
start = time.clock()
documents = []
walk = os.walk('train')
for root ,dirs, files in walk:
    for name in files:
        #documents.append([])
        filename = os.path.join(root, name)
        try:
            f = open(filename)
            #f = codecs.open(filename,'r','utf-8','ignore')
            text = f.read()
        except:
            f = codecs.open(filename,'r','gbk','ignore')    
            text = f.read()
        finally:
            f.close()
        
        documents.append(text)
#documents = news_dataset.data

print ("In the dataset there are", len(documents), "textual documents")
end = time.clock()
print('Running time: %s Seconds'%(end-start))
#print ("And this is the first one:\n", documents[0])

In the dataset there are 10746 textual documents
Running time: 33.81098931955416 Seconds


In [3]:
import gensim
import time
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
start = time.clock()        
def load_stopwords():
    f_stop = open('stopwords.txt')
    try:
        f_stop_text = f_stop.read()

    finally:
        f_stop.close()

    f_stop_seg_list = f_stop_text.split('\n')
    return f_stop_seg_list

def tokenize(text):
    stopwords = load_stopwords()    

    return [token for token in jieba.cut(text) if token.strip() not in stopwords and len(token.strip()) > 1]

#print ("After the tokenizer, the previous document becomes:\n", tokenize(documents[0]))

processed_docs = [tokenize(doc) for doc in documents]
word_count_dict = gensim.corpora.Dictionary(processed_docs)
print ("In the corpus there are", len(word_count_dict), "unique tokens")

end = time.clock()
print('Running time: %s Seconds'%(end-start))

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\dell\AppData\Local\Temp\jieba.cache
DEBUG:jieba:Loading model from cache C:\Users\dell\AppData\Local\Temp\jieba.cache
Loading model cost 1.445 seconds.
DEBUG:jieba:Loading model cost 1.445 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


In the corpus there are 216313 unique tokens
Running time: 407.2669569576021 Seconds


In [4]:
word_count_dict.filter_extremes(no_below=20, no_above=0.1) # word must appear >10 times, and no more than 20% documents
print ("After filtering, in the corpus there are only", len(word_count_dict), "unique tokens")

bag_of_words_corpus = [word_count_dict.doc2bow(pdoc) for pdoc in processed_docs]

bow_doc1 = bag_of_words_corpus[0]

#print ("Bag of words representation of the first document (tuples are composed by token_id and multiplicity):\n", bow_doc1)
print
for i in range(5):
    print ("In the document, topic_id {} (word \"{}\") appears {} time[s]".format(bow_doc1[i][0], word_count_dict[bow_doc1[i][0]], bow_doc1[i][1]))
print ("...")

After filtering, in the corpus there are only 15697 unique tokens
In the document, topic_id 91 (word "方便") appears 1 time[s]
In the document, topic_id 142 (word "心理") appears 1 time[s]
In the document, topic_id 150 (word "单一") appears 1 time[s]
In the document, topic_id 204 (word "原始") appears 1 time[s]
In the document, topic_id 212 (word "回复") appears 1 time[s]
...


In [4]:
# LDA mono-core

lper = []
topiclist=[10,50,100,150,200]
for ntopic in topiclist:
    start = time.clock()
    lda_model = gensim.models.LdaModel(bag_of_words_corpus, num_topics=10, iterations=50,id2word=word_count_dict, passes=5)
    lper.append(lda_model.log_perplexity(bag_of_words_corpus))
    end = time.clock()
    print('Running time: %s Seconds'%(end-start))
print(lper)
# LDA multicore (in this configuration, defaulty, uses n_cores-1)
# lda_model = gensim.models.LdaMulticore(bag_of_words_corpus, num_topics=10, id2word=word_count_dict, passes=5)

Running time: 264.9332727682497 Seconds
Running time: 274.0514030067159 Seconds
Running time: 269.13075202375103 Seconds
Running time: 281.0575423028499 Seconds
Running time: 278.3851060798943 Seconds
[-8.4073949386240194, -8.4535082181200831, -8.4753371336751471, -8.4235092637521234, -8.4105535283007384]


In [ ]:
x = [10,50,100,150,200]
y = [-8.4073949386240194, -8.4535082181200831, -8.4753371336751471, -8.4235092637521234, -8.4105535283007384]
print("num_topics: ",x)
print("log_perplexity: ",y)

In [8]:
import numpy as np
import matplotlib.pyplot as plt
x = topiclist
y = lper
plt.plot(x,y)
plt.xlabel('num_topics')
plt.ylabel('log_perplexity')
plt.show()


In [5]:
# LDA mono-core

lper2 = []
iteration =[50,100,500,1000,2000]
for iternum in iteration:
    start = time.clock()
    lda_model = gensim.models.LdaModel(bag_of_words_corpus, num_topics=100, iterations=iternum,id2word=word_count_dict, passes=5)
    lper2.append(lda_model.log_perplexity(bag_of_words_corpus))
    end = time.clock()
    print('Running time: %s Seconds'%(end-start))
print(lper2)

Running time: 395.0832628952928 Seconds
Running time: 407.13549432725745 Seconds
Running time: 453.2519036744234 Seconds
Running time: 433.957688734607 Seconds
Running time: 440.8043389285567 Seconds
[-8.3791937643393357, -8.4174283251262967, -8.4782422181183055, -8.4869577030165431, -8.4742420426931737]


In [ ]:
x =[50,100,500,1000,2000]
y =[-8.3791937643393357, -8.4174283251262967, -8.4782422181183055, -8.4869577030165431, -8.4742420426931737]
import numpy as np
import matplotlib.pyplot as plt
x = topiclist
y = lper
plt.plot(x,y)
plt.xlabel('num_topics')
plt.ylabel('log_perplexity')
plt.show()

In [9]:
#LDA

start = time.clock()
lda_model = gensim.models.LdaModel(bag_of_words_corpus, num_topics=100, iterations=1000,id2word=word_count_dict, passes=5)
end = time.clock()
print('Running time: %s Seconds'%(end-start))

Running time: 302.9477119165567 Seconds


 __init__(self, corpus=None, num_topics=100, id2word=None, distributed=False, chunksize=2000, passes=1, update_every=1, alpha='symmetric', eta=None, decay=0.5, offset=1.0, eval_every=10, iterations=50, gamma_threshold=0.001, minimum_probability=0.01)


In [7]:
_ = lda_model.print_topics(-1)
_

[(0,
  '0.021*孩子 + 0.019*知识 + 0.014*喜欢 + 0.011*压力 + 0.010*家庭 + 0.010*努力 + 0.008*生命 + 0.008*理解 + 0.007*兴趣 + 0.007*自信'),
 (1,
  '0.043*有限公司 + 0.030*股份 + 0.026*股东 + 0.021*股权 + 0.017*投资 + 0.016*论坛 + 0.014*深圳 + 0.014*集团 + 0.014*亿元 + 0.012*万元'),
 (2,
  '0.049*采购 + 0.048*出口 + 0.041*北约 + 0.037*综合 + 0.028*战时 + 0.027*预警 + 0.026*国产 + 0.022*任务 + 0.021*通信 + 0.021*该机'),
 (3,
  '0.060*搜索 + 0.051*com + 0.044*www + 0.037*搜狗 + 0.036*互联网 + 0.033*sogou + 0.030*网站 + 0.025*行业 + 0.018*注册 + 0.017*律师'),
 (4,
  '0.178*日本 + 0.009*人体 + 0.008*危机 + 0.008*击败 + 0.007*日本政府 + 0.007*解释 + 0.007*中日 + 0.007*&# + 0.006*失败 + 0.006*威胁'),
 (5,
  '0.053*战争 + 0.021*力量 + 0.016*民族 + 0.013*精神 + 0.011*皇帝 + 0.011*威胁 + 0.011*时代 + 0.010*传统 + 0.008*背景 + 0.008*地位'),
 (6,
  '0.066*合同 + 0.057*协议 + 0.027*签订 + 0.025*签署 + 0.024*名称 + 0.018*II + 0.018*依据 + 0.017*打赢 + 0.015*支付 + 0.015*签证'),
 (7,
  '0.024*调查 + 0.015*科目 + 0.014*农民 + 0.013*农村 + 0.010*解决 + 0.009*命令 + 0.008*锻炼 + 0.007*方法 + 0.007*企图 + 0.007*动员'),
 (8,
  '0.197*职业 + 0.037*高原 + 0.036*西安

In [18]:
for index, score in sorted(lda_model[bag_of_words_corpus[3]], key=lambda tup: -1*tup[1]):
    print ("Score: {}\t Index: {}\t \nTopic: {}".format(score, index, lda_model.print_topic(index, 5)))

Score: 0.47653010263610995	 Index: 61	 
Topic: 0.120*集团 + 0.036*轻型 + 0.035*毫米 + 0.032*集团公司 + 0.024*增长
Score: 0.21983220847303767	 Index: 2	 
Topic: 0.032*行情 + 0.026*论坛 + 0.023*上涨 + 0.016*封锁 + 0.015*价格
Score: 0.15082194075399588	 Index: 85	 
Topic: 0.072*炸弹 + 0.060*一枚 + 0.043*空投 + 0.041*攻关 + 0.041*盟友
Score: 0.08191578188618771	 Index: 88	 
Topic: 0.035*保护 + 0.029*携带 + 0.026*条例 + 0.023*高原 + 0.021*航线
Score: 0.030799234045441475	 Index: 36	 
Topic: 0.109*美元 + 0.049*密码 + 0.035*后人 + 0.035*批次 + 0.030*英特尔
Score: 0.0194951135374893	 Index: 24	 
Topic: 0.169*旅游 + 0.058*游客 + 0.037*五一 + 0.029*黄金周 + 0.022*景区
Score: 0.01111582274936802	 Index: 99	 
Topic: 0.064*股东 + 0.046*股份 + 0.041*股权 + 0.024*公告 + 0.023*有限公司


In [7]:
print ("Log perplexity of the model is", lda_model.log_perplexity(bag_of_words_corpus))

Log perplexity of the model is -8.42597499017


C000007	汽车
C000008	财经
C000010	IT
C000013	健康
C000014	体育
C000016	旅游
C000020	教育
C000022	招聘
C000023	文化
C000024	军事


In [19]:
import logging
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.WARNING)
logging.root.level = logging.WARNING
import os
import jieba
import codecs
import time
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

start = time.clock()
def read_documents(file):
    documents = []
    walk = os.walk(file)
    for root ,dirs, files in walk:
        for name in files:
            filename = os.path.join(root, name)
            try:
                f = open(filename)
                #f = codecs.open(filename,'r','utf-8','ignore')
                text = f.read()
            except:
                f = codecs.open(filename,'r','gbk','ignore')    
                text = f.read()
            finally:
                f.close()

            documents.append(text)
    return documents
#documents = news_dataset.data
documents = read_documents('train')
print ("In the dataset there are", len(documents), "textual documents")
#print ("And this is the first one:\n", documents[0])
        
def load_stopwords():
    f_stop = open('stopwords.txt')
    try:
        f_stop_text = f_stop.read()

    finally:
        f_stop.close()

    f_stop_seg_list = f_stop_text.split('\n')
    return f_stop_seg_list

def tokenize(text):
    stopwords = load_stopwords()    
    return [token for token in jieba.cut(text) if token.strip() not in stopwords and len(token.strip()) > 1]

#print ("After the tokenizer, the previous document becomes:\n", tokenize(documents[0]))


processed_docs = [tokenize(doc) for doc in documents]
word_count_dict = gensim.corpora.Dictionary(processed_docs)
print ("In the corpus there are", len(word_count_dict), "unique tokens")

word_count_dict.filter_extremes(no_below=20, no_above=0.1)
print ("After filtering, in the corpus there are only", len(word_count_dict), "unique tokens")

bag_of_words_corpus = [word_count_dict.doc2bow(pdoc) for pdoc in processed_docs]

#bow_doc1 = bag_of_words_corpus[0]
#print ("Bag of words representation of the first document (tuples are composed by token_id and multiplicity):\n", bow_doc1)
#print
#for i in range(5):
#    print ("In the document, topic_id {} (word \"{}\") appears {} time[s]".format(bow_doc1[i][0], word_count_dict[bow_doc1[i][0]], bow_doc1[i][1]))
#print ("...")


# LDA mono-core
lda_model = gensim.models.LdaModel(bag_of_words_corpus, num_topics=100,iterations=1000, id2word=word_count_dict, passes=5)

# LDA multicore (in this configuration, defaulty, uses n_cores-1)
# lda_model = gensim.models.LdaMulticore(bag_of_words_corpus, num_topics=10, id2word=word_count_dict, passes=5)

#_ = lda_model.print_topics(-1)


end = time.clock()
print('Running time: %s Seconds'%(end-start))
    

In the dataset there are 10746 textual documents
In the corpus there are 216313 unique tokens
After filtering, in the corpus there are only 15697 unique tokens
Running time: 775.6415254519088 Seconds


In [36]:
start = time.clock()
#for index, score in sorted(lda_model[bag_of_words_corpus[0]], key=lambda tup: -1*tup[1]):
#    print ("Score: {}\t Index: {}\t \nTopic: {}".format(score, index, lda_model.print_topic(index, 5)))


unseen_documents = read_documents('test')
print ("In the test dataset there are", len(unseen_documents), "unseen textual documents")

processed_docs = [tokenize(doc) for doc in unseen_documents]
#word_count_dict = gensim.corpora.Dictionary(processed_docs)
#print ("In the corpus there are", len(word_count_dict), "unique tokens")

#word_count_dict.filter_extremes(no_below=20, no_above=0.1)
#print ("After filtering, in the corpus there are only", len(word_count_dict), "unique tokens")


bow_vector = [word_count_dict.doc2bow(pdoc) for pdoc in processed_docs]


end = time.clock()
print('Running time: %s Seconds'%(end-start))    

In the test dataset there are 7164 unseen textual documents
Running time: 344.3479989507159 Seconds


In [58]:
for index, score in sorted(lda_model[bow_vector[1204]], key=lambda tup: -1*tup[1]):
    print ("Score: {}\t Index: {}\t \nTopic: {}".format(score, index,lda_model.print_topic(index, 5)))


Score: 0.42451921271739806	 Index: 49	 
Topic: 0.058*航母 + 0.048*香港 + 0.019*销售 + 0.018*西安 + 0.016*杨琳
Score: 0.09377513906361076	 Index: 1	 
Topic: 0.032*以色列 + 0.021*全球 + 0.015*报告 + 0.015*联合国 + 0.014*发言人
Score: 0.0857502936132435	 Index: 2	 
Topic: 0.047*网络 + 0.032*合作 + 0.031*网站 + 0.030*情报 + 0.023*平台
Score: 0.07960287410114551	 Index: 40	 
Topic: 0.038*集团 + 0.027*增长 + 0.020*航空 + 0.020*集团公司 + 0.020*行业
Score: 0.04509019707217458	 Index: 67	 
Topic: 0.046*股东 + 0.034*股份 + 0.029*股权 + 0.024*有限公司 + 0.021*应届
Score: 0.039585804385504186	 Index: 16	 
Topic: 0.109*台湾 + 0.069*大陆 + 0.025*军方 + 0.022*发动机 + 0.016*国民党
Score: 0.037664175452381515	 Index: 93	 
Topic: 0.113*调查 + 0.067*分钟 + 0.020*比例 + 0.016*受访者 + 0.016*37
Score: 0.034316948851955756	 Index: 63	 
Topic: 0.026*委员 + 0.022*执行 + 0.022*美日 + 0.020*委员会 + 0.019*申请
Score: 0.03371176307582605	 Index: 84	 
Topic: 0.075*战车 + 0.053*搜狐 + 0.045*体育 + 0.032*官方 + 0.030*赛季
Score: 0.03351671202853204	 Index: 56	 
Topic: 0.104*主持人 + 0.098*革命 + 0.082*青年 + 0.061*网友

In [38]:
t = lda_model.print_topics(-1)
t

[(0,
  '0.043*17 + 0.042*18 + 0.039*22 + 0.039*14 + 0.038*16 + 0.035*13 + 0.033*19 + 0.030*25 + 0.028*21 + 0.024*35'),
 (1,
  '0.032*以色列 + 0.021*全球 + 0.015*报告 + 0.015*联合国 + 0.014*发言人 + 0.013*布什 + 0.012*美元 + 0.010*伊朗核 + 0.010*宣布 + 0.010*政府'),
 (2,
  '0.047*网络 + 0.032*合作 + 0.031*网站 + 0.030*情报 + 0.023*平台 + 0.019*指挥所 + 0.017*互联网 + 0.015*网上 + 0.013*IT + 0.012*业务'),
 (3,
  '0.081*演习 + 0.022*战斗力 + 0.022*军演 + 0.021*军事演习 + 0.018*司令部 + 0.017*国防部 + 0.017*飞行员 + 0.014*隐形 + 0.013*舰队 + 0.011*印度洋'),
 (4,
  '0.139*导弹 + 0.074*俄罗斯 + 0.053*系统 + 0.026*战略 + 0.019*弹道导弹 + 0.017*防空 + 0.017*火炮 + 0.016*防御 + 0.015*毫米 + 0.012*射程'),
 (5,
  '0.114*战争 + 0.041*核武器 + 0.035*力量 + 0.020*变革 + 0.020*时代 + 0.016*快乐 + 0.015*冲突 + 0.015*世纪 + 0.014*方向 + 0.014*武力'),
 (6,
  '0.056*05 + 0.051*04 + 0.044*数学 + 0.042*营养 + 0.029*75 + 0.021*奖学金 + 0.020*01 + 0.019*大赛 + 0.019*维生素 + 0.018*成绩'),
 (7,
  '0.068*斩首 + 0.043*太阳 + 0.031*测量 + 0.025*带领 + 0.019*不仅如此 + 0.017*遭到 + 0.017*淘汰 + 0.016*首长 + 0.016*高层 + 0.014*地面'),
 (8,
  '0.059*武器 + 0.039*军事

In [43]:
print (lda_model[bow_vector])


In [55]:
for index, score in sorted(lda_model[bow_vector[2089]], key=lambda tup: -1*tup[1]):
    print ("Score: {}\t Index: {}\t \nTopic: {}".format(score, index,lda_model.print_topic(index, 5)))

Score: 0.2596375225121668	 Index: 62	 
Topic: 0.025*制度 + 0.017*改革 + 0.015*实行 + 0.014*质量 + 0.013*规范
Score: 0.16815571676100544	 Index: 33	 
Topic: 0.026*万元 + 0.019*收入 + 0.018*工资 + 0.017*贷款 + 0.013*人民币
Score: 0.10747292651282023	 Index: 64	 
Topic: 0.016*炸弹 + 0.016*城市 + 0.014*反恐 + 0.012*封锁 + 0.012*区域
Score: 0.09301956756022667	 Index: 99	 
Topic: 0.085*and + 0.052*拦截 + 0.051*万人 + 0.037*if + 0.031*威慑
Score: 0.07706781949469757	 Index: 18	 
Topic: 0.103*雷达 + 0.024*实验 + 0.022*数据 + 0.019*收费 + 0.018*系统
Score: 0.04245851802295758	 Index: 97	 
Topic: 0.068*美军 + 0.026*海上 + 0.015*空中 + 0.014*军队 + 0.013*海域
Score: 0.038905014053826056	 Index: 63	 
Topic: 0.026*委员 + 0.022*执行 + 0.022*美日 + 0.020*委员会 + 0.019*申请
Score: 0.03831571249242904	 Index: 14	 
Topic: 0.042*手机 + 0.038*短信 + 0.027*广州 + 0.025*推出 + 0.018*价格
Score: 0.03603759552054761	 Index: 66	 
Topic: 0.136*法律 + 0.062*方案 + 0.051*律师 + 0.050*独立 + 0.031*安排
Score: 0.025780127041983233	 Index: 6	 
Topic: 0.056*05 + 0.051*04 + 0.044*数学 + 0.042*营养 + 0.029*

In [50]:

with open("21.txt") as f:
    unseen_document = f.read()

print ("The unseen document is composed by the following text:", unseen_document)
print

bow_vector1 = word_count_dict.doc2bow(tokenize(unseen_document))
for index, score in sorted(lda_model[bow_vector1], key=lambda tup: -1*tup[1]):
    print ("Score: {}\t Index: {}\t \nTopic: {}".format(score, index,lda_model.print_topic(index, 5)))


The unseen document is composed by the following text: 　　本报记者王珍发自广州
　　河北沧州献县的家电经销商王凤恩，2005年年中将自己的小店改为“幸福树电器连锁店”。“大卖场步步紧逼，员工素质低，信息闭塞，制造商吸钱。”王凤恩说，这四大困难使他选择了加盟幸福树。
　　现在，已经有超过180个像王凤恩这样的农村家电经销商，成了由TCL控股的“幸福树电器连锁”的加盟商。而在浙江，也有100个这样的农村家电经销商，投入了创维加盟店的“怀抱”。
　　王凤恩们没想到的是，在“建设社会主义新农村”的背景下，在家电厂商的利益拉锯中，他们这些偏安一隅的乡镇小商人，已变为被争夺的资源，因为家电制造商们正在抢建“第三渠道”。
　　开辟“第三渠道”
　　如果说去年家电厂对于自己涉足渠道还遮遮掩掩，那么，今年它们已经不怕公开亮相了。
　　3月30日，幸福树打破缄默，在北京高调提出，今年将开发云、贵、川、晋、陕等14个省，加盟店增加到800家；以“一县一店”的规划，三年之内在全国建2000家店，形成对三、四级市场的完全覆盖。
　　创维则把“试水”范围，圈定在浙江省发达的乡镇。创维浙江、福建分部总经理黄心仲向《第一财经(相关:理财 证券)日报》透露，自2005年12月以来，创维加盟店已发展到100家，分布在浙江100个镇上。“今年的目标是开到200家。浙江省境内共有1600个镇，按一镇一店的计划，尽快覆盖500~600个镇是有可能的。”
　　为什么要开这些店呢？用创维集团副总裁、彩电事业部总裁杨东文的话是“开辟‘第三渠道’”，在大卖场和传统经销商之外，寻求“渠道创新”。
　　经过调研，创维发现，一、二级市场的机会已经不大。黄心仲说，从最近永乐大中的合并可以看出，一、二级市场的家电渠道已经基本成熟，很难再产生新渠道。相比之下，三、四级市场正在发展，尤其四级乡镇市场还有很大机会。
　　今年国家提出建设“社会主义新农村”，“中国9.5亿农民的购买力可是一座金矿！”黄心仲说，农村市场消费分散；业态落后，夫妻店居多；假冒伪劣产品横行，产品不新，价格却平均比城市贵10%，老百姓渴望信誉佳、运作好的商店。所以，“这里极有可能形成新的商圈”。
　　幸福树电器连锁的总经理张付民也直言不讳：三、四级市场占全国市场半壁江山，且每年保持15%的增长；而三、四级